# Neural Net observatory

In [1]:
%load_ext autoreload
%autoreload 2
#%matplotlib inline
%matplotlib widget

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rcParams
rcParams['figure.max_open_warning'] = 0
import ipywidgets as widgets

Fetch our tools:

In [3]:
from nn import Network, Layer, IdentityLayer, AffineLayer, MapLayer
from nnbench import NNBench
from nnvis import NNVis

Use [`ipywidgets`](https://ipywidgets.readthedocs.io/en/latest/index.html)

# Multiprocessing
We run the net training in a child process, so that it can proceed while we observe and analyze partial results.

### Tooling
 * `JSONConn` over the Process Pipe
 -- Not seeing the exception on `recv()` of a closed connection, so we accomplish a close by a non-JSON message of four bytes of zero

In [4]:
#from multiprocessing import Process, Pipe
import multiprocessing as mp
import json
from time import sleep

class JSONConn():
    def __init__(self, conn):
        self.conn = conn
        
    def send(self, v):
        self.conn.send_bytes(json.dumps(v).encode('utf8'))
        
    def poll(self):
        return self.conn.poll()
    
    def recv(self):
        r = self.conn.recv_bytes()
        if r == bytes(4):
            self.close()
            raise EOFError
        return json.loads(r)
        
    def close(self):
        self.conn.send_bytes(bytes(4))
        self.conn.close()

### The child

In [5]:
def f(conn):
    jc = JSONConn(conn)
 
    net = Network()
    net.extend(AffineLayer(2,2))
    net.extend(MapLayer(np.tanh, lambda d: 1.0 - np.tanh(d)**2))
    net.extend(AffineLayer(2,1))
    net.extend(MapLayer(np.tanh, lambda d: 1.0 - np.tanh(d)**2))

    training_batch = (np.array([[-0.5, -0.5],
                                [-0.5,  0.5],
                                [ 0.5,  0.5],
                                [ 0.5, -0.5]]),
                      np.array([[-0.5],
                                [ 0.5],
                                [-0.5],
                                [ 0.5]]))

    batch_ctr = 0
    batch_to = 0
    report_state = True
    done = False

    for i in range(100):
        if done:
            break
    #while not done:
        txm = dict()
        
        # Check for new instructions
        while jc.poll():
            rxm = jc.recv()
            print(rxm)
            for k,v in rxm.items():
                if k == 'eta':
                    net.eta = v
                elif k == 'batch to':
                    batch_to = v
                elif k == 'tell state':
                    report_state = True
                elif k == 'shutdown':
                    done = True
        
        # Report states if it's the right batch phase, or if asked to
        report_state = report_state or batch_ctr % 4 == 0 and last_state_report_at_batch < batch_ctr

        if report_state:
            txm['eta'] = [batch_ctr, net.eta]
            txm['sv'] = [batch_ctr, list(float(v) for v in net.state_vector())]
            last_state_report_at_batch = batch_ctr
            report_state = False
            
        # Run a learning step if we aren't at the target number of steps
        if batch_to > batch_ctr:
            loss = net.learn([training_batch])
            batch_ctr += 1
            txm['loss'] = [batch_ctr, loss]
        else:
            time.sleep(0.05)
            
        jc.send(txm)

    jc.close()

### The parent initiates

In [6]:
import ipywidgets as widgets
from collections import defaultdict
import rx
from rx import Observable
from rx.subject import Subject
from rx import operators as op
import time

In [7]:
if __name__ == '__main__':
    def chew(time_limit):
        print("chew")
        time.sleep(time_limit)

    # Process worker messages into topic observables
    worker_messages_s = rx.subject.Subject()
    burst_messages_s = worker_messages_s.pipe(
        op.flat_map(lambda m: m.items()))
    loss_s = burst_messages_s.pipe(
        op.filter(lambda t: t[0] == 'loss'))
    sv_s = burst_messages_s.pipe(
        op.filter(lambda t: t[0] == 'sv'))
    eta_s = burst_messages_s.pipe(
        op.filter(lambda t: t[0] == 'eta'))

    sv_s.subscribe(print) # Look at one for fun
    loss_s.pipe(op.take_last(1)).subscribe(print) # show the last loss
    

    ipc_pipe = mp.Pipe()
    parent_conn, child_conn = ipc_pipe
    jc = JSONConn(parent_conn)
    p = mp.Process(target=f, args=(child_conn,))
    p.start()
    
    jc.send({'batch to': 100})
    for i in range(110):        
        if jc.poll():
            try:
                m = jc.recv()
                worker_messages_s.on_next(m)
            except EOFError:
                worker_messages_s.on_completed()
                print("sender closed")
                break
        else:
            chew(0.1)

    p.join()


{'batch to': 100}
chew
('sv', [0, [-0.3728387052341444, -0.09801354970723603, -0.7005016261076729, 1.3411098798754615, 1.0158249836912772, -1.508880178817357, -0.8752945574046235, 2.093006574996829, -0.24808196771547272]])
('sv', [4, [-0.37238403348388394, -0.10066796409439989, -0.7027799652484302, 1.346520561584872, 1.0103437699937645, -1.4976839450178097, -0.8677653610052133, 2.084771579826037, -0.23664341413134427]])
('sv', [8, [-0.37186329802939616, -0.10363667818434485, -0.7052542666235883, 1.352493332759163, 1.004418816883815, -1.4853370721158858, -0.8596747011710051, 2.075973272060967, -0.22427937250887098]])
('sv', [12, [-0.3712605063458021, -0.10699219885777922, -0.7079612263127412, 1.3591467944422961, 0.997969518896574, -1.4715980054070132, -0.8509241664996874, 2.066523266476888, -0.21082040843146377]])
('sv', [16, [-0.3705537696053896, -0.1108334150090276, -0.7109489153216738, 1.3666397575228004, 0.9908907552104842, -1.456143547245809, -0.8413859985831299, 2.0563100667109517

---

In [8]:
assert False, "stop here if entering from above"

AssertionError: stop here if entering from above

## UI using `asyncio`

In [ ]:
%gui asyncio

In [ ]:
import asyncio
def wait_for_change(widget, value):
    future = asyncio.Future()
    def getvalue(change):
        # make the new value available
        future.set_result(change.new)
        widget.unobserve(getvalue, value)
    widget.observe(getvalue, value)
    return future

In [ ]:
from ipywidgets import IntSlider, Output
slider = IntSlider()
out = Output()

In [ ]:
async def f():
    for i in range(10):
        out.append_stdout('did work ' + str(i) + '\n')
        x = await wait_for_change(slider, 'value')
        out.append_stdout('async function continued with value ' + str(x) + '\n')
asyncio.ensure_future(f())

slider

In [ ]:
out

In [ ]:
2+2

In [ ]:
slider.value

In [ ]:
import threading
from IPython.display import display
import ipywidgets as widgets
import time
progress = widgets.FloatProgress(value=0.0, min=0.0, max=1.0)

def work(progress):
    total = 100
    for i in range(total):
        time.sleep(0.2)
        progress.value = float(i+1)/total

thread = threading.Thread(target=work, args=(progress,))
display(progress)
thread.start()

In [ ]:
i=2

In [ ]:
i+1

In [ ]:
import multiprocessing
multiprocessing.cpu_count()